# Diplomatura Universitaria en Inteligencia Artificial
## Trabajo Práctico del Módulo de Procesamiento de Lenguaje Natural

Relizado en conjunto:
[Nuria Sarochar](https://www.linkedin.com/in/nuria-sarochar/)
y
[Damián Cravacuore](https://www.linkedin.com/in/cravacuore/).

Agosto 2020.

# Imports

In [ ]:
pip install spacy

In [ ]:
import pandas as pd
import spacy
import numpy as np
from collections import Counter

# Carga del dataset

Importamos el archivo json y lo guardamos en un DataFrame pandas. Tomamos 3 columnas:

  *   'id': para poder relacionar el comentario en caso que se quiera con respecto a las conversaciones.
  *   'body': trae el texto del mensaje; a diferencia de body_html que trae el mensaje con los tags html innecesarios. Nos ahorra un paso de preprocesamiento.
  *   'is_hate': es la clasificación de los mensajes, categorizándolos como hate (1.0) o no hate (0.0).

In [ ]:
comments_file_url='https://drive.google.com/uc?id=1zWHsNwBQR2FdX75B8rtsyw5_D9NwQ0E3'
# authors_file_url='https://drive.google.com/uc?id=1zLAaZk0_ntZX1SBxsv19NBdwN09ACH7F'
# conversation_file_url='https://drive.google.com/uc?id=1Eod7JXSn4OY6cuoK2rtrb7n-y7IfABr0'

# Lectura de dato
jsondf = pd.read_json(comments_file_url)
data = jsondf.copy()

# Filtrado de dato obteniendo id de mensaje, contenido y clasificación
data = data[["id", "body", "is_hate"]]
# Convertimos Float a Int (binario)
data.is_hate = data.is_hate.astype("Int64")

print(data)

            id                                               body  is_hate
0      e8q18lf  A subsection of retarded Hungarians? Ohh boy. ...        1
1      e8q9w5s  Hiii. Just got off work. 444 is mainly the typ...        0
2      e8qbobk  wow i guess soyboys are the same in every country        0
3      e8qfn91  Owen Benjamin's soyboy song goes for every cou...        0
4      e84rl2i  wouldn't the defenders or whatever they are as...        1
...        ...                                                ...      ...
18865  dwp8jut  OP, stop being a faggot and post videos next t...        1
18866  e76148j  In this 20 minute long video, Top Hate and Cha...        1
18867  e76uzdz  No clue whos these e-celebs are, but at this p...        0
18868  e772r42      I didn’t insult you, why would you insult me?        0
18869  e77myi6                      Because you are living a lie.        0

[18870 rows x 3 columns]


Elegimos los siguientes atributos:
  * 'id' -> porque referencia al comentario dentro de una conversación.
  * 'body' -> ya que contiene el texto del mensaje en cuestión (preferimos el body antes que el 'body_html' ya que nos evita un paso de preprocesamiento para remover *tags html*)
  * 'is_hate' -> ya que nos define la clases, si es *hate* o no.

# Cálculo de estadísticas

## Detección de nulls

In [ ]:
# Detectamos que nuestra clase contiene algunos NaN
data_is_null = data[data.is_hate.isnull()]
print("-------------------UNIQUES--------------------")
print(data.is_hate.unique())
print("--------------MENSAJES SIN CLASE--------------")
print(data_is_null)
print("-------------------CANTIDAD-------------------")
print(len(data_is_null))

-------------------UNIQUES--------------------
<IntegerArray>
[1, 0, <NA>]
Length: 3, dtype: Int64
--------------MENSAJES SIN CLASE--------------
            id                                               body  is_hate
1113   e94l16e                                        What a cunt     <NA>
3383   e02y8p0                                          Dumb cunt     <NA>
4732   e9c9w2u                                   r/okbuddyretard      <NA>
5668   e9c9w2u                                   r/okbuddyretard      <NA>
5981   e94l16e                                        What a cunt     <NA>
10857  e8apbsx  Eye for an eye works only within a society or ...     <NA>
10990  e8apbsx  Eye for an eye works only within a society or ...     <NA>
12225  e02y8p0                                          Dumb cunt     <NA>
15393  e9ai3dh  How to make a meme spicy: pour some fucking Ta...     <NA>
16681  e9ai3dh  How to make a meme spicy: pour some fucking Ta...     <NA>
-------------------CANTIDAD--

## Separamos nuestro dataframe por clase

In [ ]:
data_is_hate = data[data.is_hate == 1].copy()
data_not_is_hate = data[data.is_hate == 0].copy()

## Tokenizamos y generación de estadísticas

In [ ]:
nlp = spacy.load('en_core_web_sm')
first_n_items = 1000 # Cantidad de tokens en chunk para prueba

# Aplica nlp para todos los tokens
def get_nlp_tokens(messages):
  tokens = []
  for message in messages:
    [ tokens.append(token) for token in nlp(message) ]
  return tokens

all_tokens = get_nlp_tokens(data.body[0:first_n_items])
print(all_tokens)

is_hate_tokens = get_nlp_tokens(data_is_hate.body[0:first_n_items])
not_is_hate_tokens = get_nlp_tokens(data_not_is_hate.body[0:first_n_items])

[a, subsection, of, retarded, hungarians, ?, ohh, boy, ., brace, for, a, livid, bulbasaur, coming, in, here, trying, to, hate, a, hole, in, some, of, her, stupider, countrymen, ., hiii, ., just, got, off, work, ., 444, is, mainly, the, typa, guys, you, imagine, writing, for, us, stupid, sites, ,, but, basically, they, just, try, to, fit, in, with, the, western, european, and, us, big, city, hipsters, .,  , rich, kids, from, budapest, who, feel, they, are, brave, journalistic, heroes, .,  , wow, i, guess, soyboys, are, the, same, in, every, country, owen, benjamin, 's, soyboy, song, goes, for, every, country, and, that, 's, why, it, 's, so, amazing, ., would, n't, the, defenders, or, whatever, they, are, as, a, group, be, the, most, diverse, group, ?, since, you, know, it, has, a, blind, dude, ,,  , orphan, ,, black, man, ,, woman,  , and, all, of, their, supporting, cast, ?, &, #, x200b;i, also, believe, all, the, marvel, stuff, is, going, to, get, canceled, on, netflix, because, next,

In [ ]:
def is_word(token):
  return token.is_alpha # spacy token attr para alfanuméricos

def get_words(tokens):
  return [ token for token in tokens if is_word(token) ]

def get_nouns(tokens):
  return [ token for token in tokens if token.pos_ == "NOUN" ]

def get_verbs(tokens):
  return [ token for token in tokens if token.pos_ == "VERB" ]

def get_words_nouns_verbs(tokens):
  words = get_words(tokens)
  nouns = get_nouns(words)
  verbs = get_verbs(words)
  return words, nouns, verbs

def count_and_print_common_words(word_type, words, amount_commons = 10):
  word_counter = Counter([ word.text for word in words ])
  common_words = f'Common {word_type}: {word_counter.most_common(amount_commons)}'
  print(common_words)

def tokens_statistics(tokensList):
  for tokens in tokensList:
    print(f"========================================={tokens['name']}========================================")
    words, nouns, verbs = get_words_nouns_verbs(tokens['tokens'])
    print(words)
    count_and_print_common_words('Words', words)
    count_and_print_common_words('Nouns', nouns)
    count_and_print_common_words('Verbs', verbs)

tokensList = [
  { 'name': 'ALL TOKENS', 'tokens': all_tokens },
  { 'name': 'HATE TOKENS', 'tokens': is_hate_tokens },
  { 'name': 'NO HATE TOKENS', 'tokens': not_is_hate_tokens }
]

tokens_statistics(tokensList)

=========================================ALL TOKENS========================================
[a, subsection, of, retarded, hungarians, ohh, boy, brace, for, a, livid, bulbasaur, coming, in, here, trying, to, hate, a, hole, in, some, of, her, stupider, countrymen, hiii, just, got, off, work, is, mainly, the, typa, guys, you, imagine, writing, for, us, stupid, sites, but, basically, they, just, try, to, fit, in, with, the, western, european, and, us, big, city, hipsters, rich, kids, from, budapest, who, feel, they, are, brave, journalistic, heroes, wow, i, guess, soyboys, are, the, same, in, every, country, owen, benjamin, soyboy, song, goes, for, every, country, and, that, why, it, so, amazing, would, the, defenders, or, whatever, they, are, as, a, group, be, the, most, diverse, group, since, you, know, it, has, a, blind, dude, orphan, black, man, woman, and, all, of, their, supporting, cast, also, believe, all, the, marvel, stuff, is, going, to, get, canceled, on, netflix, because, next

# Pre-procesamiento


In [ ]:
# Funciones auxiliares

# Definimos función para filtrar stop words de nuestros tokens
def get_non_stop_words(tokens):
  return [ token for token in tokens if not token.is_stop ]

## Pre-procesamiento para todos los tokens (independientemente de su clase)

In [ ]:
# Ponemos todas las palabras en minúsculas
data.body = data.body.apply(lambda text : text.lower())

# Aplicamos nuevamente la tokenización de nlp (al data set sin mayúsculas)
all_tokens = get_nlp_tokens(data.body[0:first_n_items])

# Tomamos las palabras de nuestros tokens, eliminando los números y símbolos.
words = get_words(all_tokens)

# Filtramos los stop words
words_not_stop = get_non_stop_words(words)

### Características léxicas


In [ ]:
# Hacemos lemmatización para obtener las raíces de las palabras
lemmas = [ token.lemma_ for token in words_not_stop ]
print(lemmas)

['subsection', 'retarded', 'hungarian', 'ohh', 'boy', 'brace', 'livid', 'bulbasaur', 'come', 'try', 'hate', 'hole', 'stupider', 'countryman', 'hiii', 'get', 'work', 'mainly', 'typa', 'guy', 'imagine', 'write', 'stupid', 'site', 'basically', 'try', 'fit', 'western', 'european', 'big', 'city', 'hipster', 'rich', 'kid', 'budapest', 'feel', 'brave', 'journalistic', 'hero', 'wow', 'guess', 'soyboy', 'country', 'owen', 'benjamin', 'soyboy', 'song', 'go', 'country', 'amazing', 'defender', 'group', 'diverse', 'group', 'know', 'blind', 'dude', 'orphan', 'black', 'man', 'woman', 'support', 'cast', 'believe', 'marvel', 'stuff', 'go', 'cancel', 'netflix', 'year', 'disney', 'launch', 'streaming', 'platform', 'want', 'character', 'show', 'inclusive', 'white', 'harvard', 'work', 'inclusive', 'accept', 'student', 'mean', 'literally', 'racist', 'asians', 'oh', 'yeah', 'lawsuit', 'go', 'benefit', 'white', 'long', 'run', 'unacceptable', 'view', 'aclu', 'hate', 'religion', 'christianity', 'understand', 's

## Pre-procesamiento para data 'is_hate'

In [ ]:
# Ponemos todas las palabras en minúsculas
data_is_hate.body = data_is_hate.body.apply(lambda text : text.lower())

# Aplicamos nuevamente la tokenización de nlp (al data set sin mayúsculas)
all_tokens = get_nlp_tokens(data_is_hate.body[0:first_n_items])

# Tomamos las palabras de nuestros tokens, eliminando los números y símbolos.
words = get_words(all_tokens)

# Filtramos los stop words
is_hate_words_not_stop = get_non_stop_words(words)

## Pre-procesamiento para data NO 'is_hate'

In [ ]:
# Ponemos todas las palabras en minúsculas
data_not_is_hate.body = data_not_is_hate.body.apply(lambda text : text.lower())

# Aplicamos nuevamente la tokenización de nlp (al data set sin mayúsculas)
all_tokens = get_nlp_tokens(data_not_is_hate.body[0:first_n_items])

# Tomamos las palabras de nuestros tokens, eliminando los números y símbolos.
words = get_words(all_tokens)

# Filtramos los stop words
not_is_hate_words_not_stop = get_non_stop_words(words)

## Re-cálculo de las estadísticas del dataset

In [ ]:
tokensList = [
  { 'name': 'PROCESSED TOKENS', 'tokens': words_not_stop },
  { 'name': 'HATE TOKENS', 'tokens': is_hate_words_not_stop },
  { 'name': 'NO HATE TOKENS', 'tokens': not_is_hate_words_not_stop },
]

tokens_statistics(tokensList)

=========================================PROCESSED TOKENS========================================
[subsection, retarded, hungarians, ohh, boy, brace, livid, bulbasaur, coming, trying, hate, hole, stupider, countrymen, hiii, got, work, mainly, typa, guys, imagine, writing, stupid, sites, basically, try, fit, western, european, big, city, hipsters, rich, kids, budapest, feel, brave, journalistic, heroes, wow, guess, soyboys, country, owen, benjamin, soyboy, song, goes, country, amazing, defenders, group, diverse, group, know, blind, dude, orphan, black, man, woman, supporting, cast, believe, marvel, stuff, going, canceled, netflix, year, disney, launching, streaming, platform, want, characters, shows, inclusive, white, harvard, working, inclusive, accepting, students, meant, literally, racist, asians, oh, yeah, lawsuit, going, benefit, whites, long, run, unacceptable, view, aclu, hate, religion, christianity, understood, sentiment, atheist, bordering, ti, theist, dislike, theistic, relig

# Pre-procesamiento aplicado

Aplicamos pre-procesamiento por mensaje a todo el dataset

In [ ]:
# Aplicamos el nlp, tomamos aquellos tokens que son palabras y de ellos los que no son stop words.
tokenizedMessages = data.body.apply(nlp).apply(get_words).apply(get_non_stop_words)

# Nos quedamos con la versión 'lemmatizada' de los tokens
tokenizedMessages_lemma = tokenizedMessages.apply(lambda message: [token.lemma_ for token in message])

print(tokenizedMessages)
print("\n\n============LEMMAS==============\n")
print(tokenizedMessages_lemma)

0        [subsection, retarded, hungarians, ohh, boy, b...
1        [hiii, got, work, mainly, typa, guys, imagine,...
2                           [wow, guess, soyboys, country]
3        [owen, benjamin, soyboy, song, goes, country, ...
4        [defenders, group, diverse, group, know, blind...
                               ...                        
18865         [op, stop, faggot, post, videos, time, hard]
18866    [minute, long, video, hate, champagne, goes, n...
18867    [clue, s, e, celebs, point, time, need, progre...
18868                                     [insult, insult]
18869                                        [living, lie]
Name: body, Length: 18870, dtype: object


============LEMMAS==============

0        [subsection, retarded, hungarian, ohh, boy, br...
1        [hiii, get, work, mainly, typa, guy, imagine, ...
2                            [wow, guess, soyboy, country]
3        [owen, benjamin, soyboy, song, go, country, am...
4        [defender, group, diverse, gr